In [440]:
import awkward as ak
import numpy as np
import uproot as up
import hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import matplotlib.pyplot as plt
import correctionlib
from coffea.jetmet_tools import FactorizedJetCorrector, JetCorrectionUncertainty
from coffea.jetmet_tools import JECStack, CorrectedJetsFactory
from coffea.lookup_tools import extractor

fname = "G-4Jets_HT-600_postEE.root"
events = NanoEventsFactory.from_root(
    fname,
    schemaclass=NanoAODSchema.v6,
    metadata={"dataset": "DYJets"},
).events()
totGenWeight = np.sum(events.genWeight)
totGenWeight

364187000.0

# Working with $\gamma$

In [441]:
photon = events.Photon
#photon.fields

In [442]:
#Applying Trigger cuts 
hlt = events.HLT
trig_cut = np.stack((hlt.Photon20, hlt.Photon33, hlt.Photon50, hlt.Photon75, hlt.Photon90, hlt.Photon120, hlt.Photon150, hlt.Photon175, hlt.Photon200,), axis=1)
photon = photon[ak.any(trig_cut, axis=1)]
events = events[ak.any(trig_cut, axis=1)]
photon.pt

<Array [[121], [82.6], ... [63, 26.6], [84]] type='563245 * var * float32[parame...'>

In [443]:
#Applying pt and eta and other quality cuts
pt_eta_cut = (ak.num(photon)>=1) & (abs(photon.eta)<2.3) & (photon.pixelSeed==0) & (photon.mvaID_WP80==True) & (photon.pfPhoIso03<1)
print(pt_eta_cut)
photon = photon[pt_eta_cut]
photon = photon[ak.num(photon)==1]
events = events[ak.num(photon)==1]
photon.pt

[[False], [False], [False], [True], ... [True, False], [False, False], [False]]


<Array [[82.1], [169], ... [245], [45.5]] type='250519 * var * float32[parameter...'>

In [451]:
deltaR = photon[:,0].delta_r(events.GenIsolatedPhoton)
deltaR
print('Sel Photon eta=', photon.eta)
print('Sel Photon phi=', photon.phi)
print('Gen Photon eta=', events.GenIsolatedPhoton.eta)
print('Gen Photon phi=', events.GenIsolatedPhoton.phi)
#print(ak.num(events.GenIsolatedPhoton))

Sel Photon eta= [[0.798], [0.182], [-1.15], [0.474], [1.59, ... [-1.61], [2.26], [-1.87], [2]]
Sel Photon phi= [[-0.523], [-2.38], [-2.11], [-1.5], ... [-0.668], [2.52], [1.86], [-1.47]]
Gen Photon eta= [[0.287], [-0.994], [1.09], [0.794], ... 0.762], [], [-0.357], [0.452], [-0.857]]
Gen Photon phi= [[0.996], [1.72], [-1.12], [-0.524], ... -2.87], [], [1.2], [-0.865], [0.294]]


In [418]:
#photon = photon[ak.num(photon)==1]
photon.pt
sel_photon = photon[:,0]

In [419]:
# Load the correctionlib JSON file #
evaluator = correctionlib.CorrectionSet.from_file("EGmSFs/SS.json")
print(list(evaluator.keys()))
# Obtain the smearing values
evaluator_smearing = evaluator["Prompt2022FG_SmearingJSON"]
rho = evaluator_smearing.evaluate("rho", sel_photon.eta, sel_photon.r9)
rng = np.random.default_rng(seed=125)  # The smearing is done statistically, so we need some random numbers
smearing = rng.normal(loc=1., scale=rho)
sel_photon_pt_smearing_nom = smearing * sel_photon.pt

['Prompt2022FG_ScaleJSON', 'Prompt2022FG_SmearingJSON']


In [420]:
# Systematic uncertainty on smearing (evaluated on MC)
unc_rho = evaluator_smearing.evaluate("err_rho", sel_photon.eta, sel_photon.r9)
smearing_up = rng.normal(loc=1., scale=rho + unc_rho)
smearing_down = rng.normal(loc=1., scale=rho - unc_rho)
sel_photon_pt_smearing_up = smearing_up * sel_photon.pt
sel_photon_pt_smearing_down = smearing_down * sel_photon.pt

In [421]:
# Obtain the scale values. The nominal scale correction is applied on data
evaluator_scale = evaluator["Prompt2022FG_ScaleJSON"]

# Systematic uncertainty for scale: Careful, this is evaluated on MC because we always apply systematics on MC, not on data
scale_MC_unc = evaluator_scale.evaluate("total_uncertainty", sel_photon.seedGain, 1.0, sel_photon.eta, sel_photon.r9, sel_photon.pt)

# To use the systematic variations in your analysis, you should treat scale_up and scale_down as multiplicative factors for the uncorrected (nominal) photon pt in MC
sel_photon_pt_scale_up = (1+scale_MC_unc) * sel_photon.pt
sel_photon_pt_scale_down = (1-scale_MC_unc) * sel_photon.pt
sel_photon_pt_scale_down

<Array [82.1, 169, 153, ... 106, 245, 45.5] type='250519 * float64'>

In [429]:
deltaR = sel_photon.delta_r(events.GenIsolatedPhoton)
deltaR[deltaR<0.5]

<Array [[], [], [], [], ... [], [], [], []] type='250519 * var * float32'>

# Jet selection

In [424]:
jet = events.Jet
ext = extractor()
ext.add_weight_sets([
    "* * EGmSFs/Summer22EE_22Sep2023_V2_MC_JEC/Summer22EE_22Sep2023_V2_MC_L2Relative_AK4PFPuppi.txt",
    "* * EGmSFs/Summer22EE_22Sep2023_V2_MC_JEC/Summer22EE_22Sep2023_V2_MC_Uncertainty_AK4PFPuppi.junc.txt",
])
ext.finalize()

jec_stack_names = [
    "Summer22EE_22Sep2023_V2_MC_L2Relative_AK4PFPuppi",
    "Summer22EE_22Sep2023_V2_MC_Uncertainty_AK4PFPuppi",
]

evaluator = ext.make_evaluator()

jec_inputs = {name: evaluator[name] for name in jec_stack_names}
jec_stack = JECStack(jec_inputs)

name_map = jec_stack.blank_name_map
name_map['JetPt'] = 'pt'
name_map['JetMass'] = 'mass'
name_map['JetEta'] = 'eta'
name_map['JetA'] = 'area'

jet['pt_raw'] = (1 - jet['rawFactor']) * jet['pt']
jet['mass_raw'] = (1 - jet['rawFactor']) * jet['mass']
jet['pt_gen'] = ak.values_astype(ak.fill_none(jet.matched_gen.pt, 0), np.float32)
#jet['rho'] = ak.broadcast_arrays(events.fixedGridRhoFastjetAll, jet.pt)[0]
name_map['ptGenJet'] = 'pt_gen'
name_map['ptRaw'] = 'pt_raw'
name_map['massRaw'] = 'mass_raw'
#name_map['Rho'] = 'rho'

events_cache = events.caches[0]
corrector = FactorizedJetCorrector(
    Summer22EE_22Sep2023_V2_MC_L2Relative_AK4PFPuppi = evaluator['Summer22EE_22Sep2023_V2_MC_L2Relative_AK4PFPuppi'],
)
uncertainties = JetCorrectionUncertainty(
    Summer22EEPrompt22_JRV1_MC_SF_AK4PFPuppi=evaluator['Summer22EE_22Sep2023_V2_MC_Uncertainty_AK4PFPuppi']
)


jet_factory = CorrectedJetsFactory(name_map, jec_stack)
corr_jets = jet_factory.build(jet, lazy_cache=events_cache)
#print('new columns:', set(ak.fields(corr_jets)) - set(ak.fields(jet)))
#print('All columns:', set(ak.fields(corr_jets)))
#corr_jets.fields

jet_sel_cut = (corr_jets.pt>30) & (abs(corr_jets.eta)<2.5) & (corr_jets.jetId==6)
#corr_jets.JES_jes
corr_jets.jet_energy_uncertainty_jes
JES_down = corr_jets.jet_energy_uncertainty_jes[:,:,0]
JES_up = corr_jets.jet_energy_uncertainty_jes[:,:,1]
pt_jec_up = corr_jets.pt_orig*JES_up
pt_jec_down = corr_jets.pt_orig*JES_down
JES_up
#corr_jets.fields
corr_jets.pt

<Array [[356, 187, 130, ... 54.3, 52.2, 21]] type='250519 * var * float32'>

# Saving different variables and Histograms in ROOT File

In [425]:
file = up.recreate('output_GJets_Ht_600.root')
#file["tree3"] = {"Photon_pt": sel_photon_pt_smearing_nom,"nJet": nJet,"isJet":jet_pt_eta_cut,"Jet_pt": jet.pt}
file["tree"] = {"Jet": ak.zip({
                        "pt": corr_jets.pt,
                        "eta": corr_jets.eta,
                        "isId": jet_sel_cut,
                        "pt_orig":corr_jets.pt_orig,
                        "pt_jec_nom": corr_jets.pt_jec,
                        "pt_jec_up":pt_jec_up,
                        "pt_jec_down":pt_jec_down,
                        #"JES_jes":corr_jets.JES_jes,
                        #"energy_uncertainty_jes": corr_jets.jet_energy_uncertainty_jes,
                        }),
                 "Photon": ak.zip({
                     "pt_nom":sel_photon_pt_smearing_nom,
                     "pt_smearing_up":sel_photon_pt_smearing_up,
                     "pt_smearing_down":sel_photon_pt_smearing_down,
                     "pt_scale_up":sel_photon_pt_scale_up,
                     "pt_scale_down":sel_photon_pt_scale_down,
                     "eta_nom":sel_photon.eta,
                     "phi_nom":sel_photon.phi,
                 })
                }
import hist
#Define the histograms
h_photon_pt_nom = hist.Hist.new.Reg(200, 0, 1000, name="photon_pt").Weight()
h_photon_eta_nom = hist.Hist.new.Reg(60, -3, 3, name="photon_eta").Weight()
h_photon_phi_nom = hist.Hist.new.Reg(70, -3.5, 3.5, name="photon_phi").Weight()

h_leadJet_pt_nom = hist.Hist.new.Reg(200, 0, 1000, name="leadJet_pt").Weight()
h_leadJet_eta_nom = hist.Hist.new.Reg(60, -3, 3, name="leadJet_eta").Weight()
h_leadJet_phi_nom = hist.Hist.new.Reg(70, -3.5, 3.5, name="leadJet_phi").Weight()
#Fill the histograms
h_photon_pt_nom.fill(sel_photon_pt_smearing_nom[sel_photon_pt_smearing_nom>20], weight=53.91*1000.0*events.genWeight[sel_photon_pt_smearing_nom>20]/totGenWeight)
h_photon_eta_nom.fill(sel_photon.eta[sel_photon_pt_smearing_nom>20], weight=1.0)
h_photon_phi_nom.fill(sel_photon.phi[sel_photon_pt_smearing_nom>20], weight=1.0)


#h_leadJet_pt_nom.fill(corr_jets[:,0].pt_jec[ak.num(corr_jets[:,0].pt_jec>30], weight=1.0)
#h_leadJet_eta_nom.fill(corr_jets[:,0].eta[corr_jets[:,0].pt_jec>30], weight=1.0)
#h_leadJet_phi_nom.fill(corr_jets[:,0].phi[corr_jets[:,0].pt_jec>30], weight=1.0)

#Save the histograms
file["Photon_pt_nom"] = h_photon_pt_nom
file["Photon_eta_nom"] = h_photon_eta_nom
file["Photon_phi_nom"] = h_photon_phi_nom

file["leadJet_pt_nom"] = h_leadJet_pt_nom
file["leadJet_eta_nom"] = h_leadJet_eta_nom
file["leadJet_phi_nom"] = h_leadJet_phi_nom

file.close()

In [ ]:
#Total Gen-Weights of Gamma+jets MC samples
#G-Jets Ht= 40 to 70 == 4.47579e+10[44694010000]    xsection = 15240*e+3[fb]
#G-Jets Ht= 70 to 100 == 3.26817e+10   xsection = 8198*e+3
#G-Jets Ht= 100 to 20 == 2.34393e+10   xsection = 7404*e+3
#G-Jets Ht= 200 to 400 == 7.51799e+09[7509937000]  xsection = 1548*e+3
#G-Jets Ht= 400 to 600 == 6.59903e+08  xsection = 166.1*e+3
#G-Jets Ht= 600 to inf == 3.6527e+08[364187000]   xsection = 53.91*e+3

In [ ]:
fig, ax_pt = plt.subplots()
ax_pt.hist(sel_photon_pt_smearing_nom[sel_photon_pt_smearing_nom>20], bins=100),plt.xlabel("$p_{T}$ [GeV]"),plt.ylabel("Events")

fig, ax_eta = plt.subplots()
ax_eta.hist(sel_photon.eta[sel_photon_pt_smearing_nom>20], bins=100), plt.xlabel("$\eta$"),plt.ylabel("Events")

fig, ax_r9 = plt.subplots()
ax_r9.hist(sel_photon.r9[sel_photon_pt_smearing_nom>20], bins=100), plt.xlabel("$R_{9}$"),plt.ylabel("Events")

fig, ax_sieie = plt.subplots()
ax_sieie.hist(photon.sieie[sel_photon_pt_smearing_nom>20], bins=100), plt.xlabel("$\sigma_{i\eta i\eta}$"),plt.ylabel("Events")

fig, ax_sieip = plt.subplots()
ax_sieip.hist(photon.sieip[sel_photon_pt_smearing_nom>20], bins=100), plt.xlabel("$\sigma_{i\eta i\phi}$"),plt.ylabel("Events")
#ax.set_xscale("log")
#ax.legend(title="Photon $p_{T}$ [GeV]")